In [1]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import convert_and_clean, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize

from time import time, sleep

from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
%load_ext line_profiler

Loading BokehJS ...

In [2]:
obs_steps = 300
period = 30
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP"]
init_funds = {"BTC":'0.00000000', "ETH":'0.00000000', "LTC":'0.00000000', "XRP":'0.00000000', "USDT":'100.00000000'}

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, pairs, init_funds)
tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
                       start=datetime.timestamp(datetime.utcnow() - timedelta(days=150)))

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'USDT'
obs = env.reset()

INFO:backtest_env:[Trading Environment initialization]
Trading Environment Initialized!



In [3]:
# Training params
nb_steps = 100
batch_size = 4
nb_max_episode_steps = 4

agent = apriori.MomentumTrader()
opt_params, info = agent.fit(env, nb_steps, batch_size, nb_max_episode_steps=nb_max_episode_steps)
print("\n", opt_params,"\n", env.status)

agent.test(env, verbose=True)
env.plot_results();

Optimizing model for 100 steps with batch size 4...
Optimization step 59/100, step reward: 0.9990737208942697, ETC: 0 days 00:01:22.470948                     
 {'mean_type': 'kama', 'alpha_down': 0.5794398478251988, 'alpha_up': 0.33446025170732036, 'ma1': 2.91223303964507, 'ma2': 219.92782417711777, 'std_span': 46.5366198393675} 
 {'OOD': False, 'Error': False, 'ValueError': False, 'ActionError': False}
>> step 2099/2099, 100 % done, Cumulative Reward: 0.767537259142989, ETC: 0 days 00:00:00                                  
################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.037482
Portifolio / Benchmark Sharpe ratio:      -0.753538 / -0.005049
Portifolio / Benchmark Omega ratio:       0.803693 / 0.999095
Portifolio / Benchmark max drawdown:      -23.276203 / -320.659900


In [4]:
tapi.download_data(end=datetime.timestamp(datetime.now()),
                       start=datetime.timestamp(datetime.now() - timedelta(days=100)))

agent.test(env, verbose=True)
env.plot_results();

>> step 4497/4497, 100 % done, Cumulative Reward: 1.0477023222336561, ETC: 0 days 00:00:00                                 
################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.033819
Portifolio / Benchmark Sharpe ratio:      -0.894195 / 0.201000
Portifolio / Benchmark Omega ratio:       0.791546 / 1.040014
Portifolio / Benchmark max drawdown:      -1.001386 / -4.857212
